In [ ]:
!pip uninstall pyfts -y
!pip install git+https://github.com/PYFTS/pyFTS
!pip install SimpSOM

  Cloning https://github.com/PYFTS/pyFTS to /tmp/pip-req-build-dgbu3c0t
  Running command git clone -q https://github.com/PYFTS/pyFTS /tmp/pip-req-build-dgbu3c0t
  Created wheel for pyFTS: filename=pyFTS-1.6-cp37-none-any.whl size=217117 sha256=eda9d77820d5730f5ef36a3db106c9044f0014e8debc08e2ecc545ff835bec78
  Stored in directory: /tmp/pip-ephem-wheel-cache-6oa7twt3/wheels/e7/32/a9/230470113df5a73242a5a6d05671cb646db97abf14bbce2644
Successfully built pyFTS
  Created wheel for SimpSOM: filename=SimpSOM-1.3.4-cp37-none-any.whl size=17088 sha256=4ca1fb942b424add9cdbe39079a2116f09813ca29d7f6fb15274f9729f59ae3f
  Stored in directory: /root/.cache/pip/wheels/51/76/ee/c02f0bc20af4cd8f46ee4142ebf7bb654b737d6d8f2360d26b
Successfully built SimpSOM


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyFTS.partitioners import Grid
from pyFTS.models import hofts, pwfts
from pyFTS.common import Util as cUtil
from pyFTS.benchmarks import Measures
from pyFTS.common.transformations.differential import Differential 

## NADAQ e S&P500

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

closing_prices = pd.read_csv('/content/drive/MyDrive/meu_mestrado/closing_price_nasdaq_sp500.csv', index_col = 0)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
closing_prices['sentiment_hesitant'] 

2016-12-28    939
2016-12-29    938
2016-12-30    925
2017-01-03    941
2017-01-04    826
             ... 
2020-05-22    753
2020-05-26    800
2020-05-27    818
2020-05-28    775
2020-05-29    749
Name: sentiment_hesitant, Length: 860, dtype: int64

In [ ]:
closing_prices['sp500_close_shift'] = closing_prices['sp500_close'].shift(1)

closing_prices['nasdaq_close_shift'] = closing_prices['nasdaq_close'].shift(1)

closing_prices['sp500_return_shift'] = closing_prices['sp500_return'].shift(1)

closing_prices['nasdaq_return_shift'] = closing_prices['nasdaq_return'].shift(1)

closing_prices['sentiment_hesitant_shift'] = closing_prices['sentiment_hesitant'].shift(1)

closing_prices['sentiment_hesitant'] = closing_prices['sentiment_hesitant']

nasdaq_mv = closing_prices[['nasdaq_close','hesitant_return','nasdaq_close_shift', 'sentiment_hesitant','sentiment_hesitant_shift']]
sp500_mv = closing_prices[['sp500_close','hesitant_return','sp500_close_shift', 'sentiment_hesitant','sentiment_hesitant_shift']]

nasdaq_mv.rename(columns = {'nasdaq_close_shift': 'close_shift'}, 
                 inplace=True)

sp500_mv.rename(columns = {'sp500_close_shift': 'close_shift'}, 
                 inplace=True)

nasdaq_mv.dropna(inplace=True)

sp500_mv.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
from pyFTS.models.multivariate import variable, mvfts, wmvfts
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyFTS.partitioners import Grid
from pyFTS.models import hofts, pwfts, chen 
from pyFTS.common import Util as cUtil
from pyFTS.benchmarks import Measures
from pyFTS.common.transformations.differential import Differential 
from copy import deepcopy


# def check_direcao (row):
#   if row['real'] >= 0 and row['forecast'] >= 0:
#     return 1 
#   elif row['real'] <= 0 and row['forecast'] <= 0:
#     return 1 
#   else:
#     return 0

def MDA(original, forecasts):
   n = len(original)
   count =0
   for t in range(1,n):
      # print(original[t], original[t-1])
      # print(forecasts[t], forecasts[t-1])

      y = 1 if float(original[t]) - float(original[t-1]) >= 0 else -1
      f = 1 if float(forecasts [t]) - float(forecasts[t-1]) >=0 else -1
      count += 1 if y == f else 0
   return count/n

# def rmse(targets, forecasts, order=0, offset=0):
#     """
#     Root Mean Squared Error
#     :param targets: array of targets
#     :param forecasts: array of forecasts
#     :param order: model order
#     :param offset: forecast offset related to target. 
#     :return: 
#     """
#     if isinstance(targets, list):
#         targets = np.array(targets)
#     if isinstance(forecasts, list):
#         forecasts = np.array(forecasts)
    
#     if offset == 0:
#         return np.sqrt(np.nanmean((targets[order:] - forecasts[:]) ** 2))
#     else:
#         return np.sqrt(np.nanmean((targets[order+offset:] - forecasts[:-offset]) ** 2))


# def mape(targets, forecasts):
#     """
#     Mean Average Percentual Error
#     :param targets: 
#     :param forecasts: 
#     :return: 
#     """
#     if isinstance(targets, list):
#         targets = np.array(targets)
#     if isinstance(forecasts, list):
#         forecasts = np.array(forecasts)
#     return np.nanmean(np.abs(np.divide(np.subtract(targets, forecasts), targets))) * 100


def run_multivariate(time_series,variables, sent_n_part, close_n_part,window_size,train_test_split):
  measurements = {'rmse':[], 'mape': [], 'u': [], 'nrmse1':[],'nrmse2':[] }
  diff = Differential(1)
  main_column = time_series.columns[0] 
  time_series = time_series[[main_column] + variables]
  lista_forecast = []
  lista_real = []
  # print(time_series)
  # print(main_column)
  for count, train, test in cUtil.sliding_window(time_series, window_size, train_test_split, 1 - train_test_split):
    # print('-1')
    variable_list = []
    for v in variables:
      # print('passou nas variavel')
      variable_list.append(variable.Variable(v, alias=v[0], npart=sent_n_part,
                                        partitioner=Grid.GridPartitioner, data=train,
                                        data_label=v))
      

    if is_diff:
      # print('os_preco')
      var_price = variable.Variable('Price', alias='P', npart=close_n_part,
                                    partitioner=Grid.GridPartitioner, data=train,
                                  transformation=diff,
                                  data_label=str(main_column))
      # print('ao o o')
      
    else:
      var_price = variable.Variable('Price', alias='P', npart=close_n_part,
                                      partitioner=Grid.GridPartitioner, data=train,
                                      data_label=main_column)
    # print([var_price] + variable_list)
    # print(0)
    # model = wmvfts.WeightedMVFTS(explanatory_variables=[var_price] + variable_list,
    #                     target_variable=var_price)
    model = mvfts.MVFTS(explanatory_variables=[var_price] + variable_list,
                        target_variable=var_price)
    model.fit(train)
    # print('1')
    # print('2')
    forecast = deepcopy(model).predict(test)
    rmse_t, mape_t, _ = deepcopy(Measures.get_point_statistics(test, deepcopy(model)))
    # print('3')
    measurements['rmse'].append( rmse_t)
    measurements['mape'].append( mape_t)
    # print('4')
    lista_forecast = lista_forecast + list(forecast)
    lista_real = lista_real + list(test[main_column])
    # print('5')
  # print(measurements['mape'], measurements['rmse'])
  # print(lista_real, lista_forecast)
  # print('6')
  mda = MDA(lista_real, lista_forecast)
  # print(mda)
  rmse_t =  mean(measurements['rmse'])
  mape_t = mean(measurements['mape'])
  variables_t = '|'.join(variables)
  # print('7')
  df_final = pd.DataFrame([[close_n_part, sent_n_part, main_column, window_size, mda,rmse_t, mape_t, variables_t]], columns = ['close_n_part', 'sent_n_part', 'index', 'window_size', 'accuracy', 'rmse', 'mape', 'variables'])
  return df_final

def run_univariate(df,n_part,window_size,train_test_split):
  measurements = {'rmse':[], 'mape': [], 'u': [], 'nrmse1':[],'nrmse2':[] }
  diff = Differential(1)
  time_series = df.iloc[:,0].values
  index = df.columns[0]
  acertos = []
  lista_forecast = []
  lista_real = []
  for count, train, test in cUtil.sliding_window(time_series, window_size, train_test_split, 1 - train_test_split):
    if is_diff:
      fs = Grid.GridPartitioner(data=train, npart=n_part, transformation=diff)
    else:
      fs = Grid.GridPartitioner(data=train, npart=n_part)
    model = chen.ConventionalFTS(partitioner=fs)
    # model = hofts.WeightedHighOrderFTS(partitioner=fs, order=1)
    if is_diff:
      model.append_transformation(diff)

    model.fit(train)
    forecast = model.predict(test)
    rmse_t, mape_t, _ = Measures.get_point_statistics(test, model)
    measurements['rmse'].append( rmse_t)
    measurements['mape'].append( mape_t)
    lista_forecast = lista_forecast + list(forecast)
    lista_real = lista_real + list(test[1:])
  # print(lista_forecast)
  # print(lista_real)
  mda = MDA(lista_real, lista_forecast)
  rmse_t =  mean(measurements['rmse'])
  mape_t = mean(measurements['mape'])
  df_final = pd.DataFrame([[close_n_part, '-', index, window_size, mda,rmse_t, mape_t, '-']], columns = ['close_n_part', 'sent_n_part', 'index', 'window_size', 'accuracy', 'rmse', 'mape', 'variables'])
  return df_final


In [ ]:

close_n_part_list = [5, 10, 15,20,25]
sent_n_part_list = [3, 5, 7]
is_diff = 1
window_size_list = [600]
ts_list = [nasdaq_mv, sp500_mv]
train_test_split = 0.95
df = pd.DataFrame()
variable_list = [ ['sentiment_hesitant']]


for close_n_part in close_n_part_list:
  for window_size in window_size_list:
    for ts in ts_list:

      df_aux = run_univariate(ts,close_n_part,window_size,train_test_split)
      df = pd.concat([df, df_aux])

      for sent_n_part in sent_n_part_list:
        print(close_n_part,window_size, sent_n_part)
        for variables in variable_list:

          df_aux = run_multivariate(ts,variables, sent_n_part, close_n_part,window_size,train_test_split)

          df = pd.concat([df, df_aux])
# run_univariate(sp500_mv, close_n_part,window_size,train_test_split)

5 600 3


/usr/local/lib/python3.7/dist-packages/pyFTS/models/multivariate/mvfts.py:165: RuntimeWarning: invalid value encountered in double_scalars
  ret.append(np.dot(mv,mp.T)/np.nansum(mv))


5 600 5
5 600 7
5 600 3
5 600 5
5 600 7
10 600 3
10 600 5
10 600 7
10 600 3
10 600 5
10 600 7
15 600 3
15 600 5
15 600 7
15 600 3
15 600 5
15 600 7
20 600 3
20 600 5
20 600 7
20 600 3
20 600 5
20 600 7
25 600 3
25 600 5
25 600 7
25 600 3
25 600 5
25 600 7
